In [ ]:
import os
import readtop2000
import tqdm
import pandas as pd
import locale

In [ ]:
URL_TOP2000_HISTORY = 'https://nl.wikipedia.org/wiki/Lijst_van_Radio_2-Top_2000%27s'

## Base tables

In [ ]:
import importlib;importlib.reload(readtop2000)
def read():
    fulllist_reader = readtop2000.WikipediaTableExtractor(URL_TOP2000_HISTORY, [1], [0])
    df = fulllist_reader.extract_table_as_dataframe()
    
    cleaner = readtop2000.Top2000Cleaner(df)
    notering, song, songartist, artist = cleaner.clean()
    return notering, song, songartist, artist
notering, song, songartist, artist = read()

There are a few artists which have the same name, but are not the same artist:
* Nena, who performed as a singer, but also as part of the band named Nena
* Mr. Big was both an American and a British band

In [ ]:
artist[artist.groupby('Name')['Link'].transform('nunique').gt(1)]

## Detailed information for artists from inboxes
Drop Space Monkey because it does have an infobox, but it is about the song, not the artist

Drop Anita Garbo because the wikipedia redirects to a song, instead of to a page about her


In [ ]:
def download_infobox_details(links):
    result = []
    for link in tqdm.tqdm(links):
        info = readtop2000.InfoboxReader(link, allow_errors=True).read()
        result.append(info)
    return pd.concat(result)

ignored_artists = ['Anita Garbo',  # Wikipedia redirects to her song, instead of a page about her as an artist
                   'Space Monkey',  # Wikipedia does have an infobox, but it is about their song, not about them as an artist
                  ]

links = artist.loc[~artist['Name'].isin(ignored_artists), 'Link']
artist_details = download_infobox_details(links)

In [ ]:
# The band members are much harder to handle because of all the functions they can have: it gives a many-to-many relation for members and bands
# So we ignore those. The same goes vice versa for the member pages in which it is discussed in what bands they were active
extra_artist_details = (artist_details[~artist_details['Header'].isin(['Leden', 'Oud-leden', 'Bezetting']) & ~artist_details['Header'].str.startswith('Actief')]  
                         .set_index(['OriginalLink', 'Variable'])['Value'].unstack())

In [ ]:
# We download from dutch wikipedia, so we need dutch month names
locale.setlocale(locale.LC_TIME, 'nl_NL.utf8')
def wikipedia_datetime_to_datetime(series):
    return pd.to_datetime(series, errors='coerce', format='%d %B %Y', exact=False)
    
extra_artist_details = extra_artist_details.assign(Overleden = lambda df: wikipedia_datetime_to_datetime(df['Overleden']),
                                                   Geboren = lambda df: wikipedia_datetime_to_datetime(df['Geboren']),
                                                  )


In [ ]:
artist_full = artist.merge(extra_artist_details, right_index=True, left_on='Link', how='left')

## Save all tables

In [ ]:
tables = {'notering': notering,
          'song': song,
          'songartist': songartist,
          'artist_small': artist,
          'artist': artist_full,
          'artist_details': artist_details
         }
for name, table in tables.items():
    table.to_parquet(os.path.join('Data', f'{name}.parquet'))